This experiment, higher learning rate. Larger rollouts.

In [1]:
import deep_rl

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
import torch
from torch.nn import functional as F
from torch.autograd import Variable
from torch import nn, optim
import torch.utils.data

# load as dask array
import time

import logging
import sys
import os
import glob
import numpy as np
import datetime
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm

In [4]:
from deep_rl.utils import Config
from deep_rl.utils.logger import get_logger, get_default_log_dir

from deep_rl.network.network_heads import CategoricalActorCriticNet, QuantileNet, OptionCriticNet, DeterministicActorCriticNet, GaussianActorCriticNet
from deep_rl.network.network_bodies import FCBody

from deep_rl.component.task import ParallelizedTask

In [5]:
from world_models_sonic.models.vae import VAE5, loss_function_vae
from world_models_sonic.helpers.summarize import TorchSummarizeDf
from world_models_sonic.helpers.dataset import load_cache_data
from world_models_sonic.models.rnn import MDNRNN2
from world_models_sonic.models.inverse_model import InverseModel
from world_models_sonic.models.world_model import WorldModel
from world_models_sonic.custom_envs.env import make_env
from world_models_sonic.custom_envs.wrappers import RandomGameReset
from world_models_sonic import config
from world_models_sonic.helpers.deep_rl import PPOAgent, run_iterations, SonicWorldModelDeepRL, CategoricalWorldActorCriticNet

Importing 0 potential games...
Imported 0 games


/home/wassname/.pyenv/versions/3.5.3/envs/jupyter3/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Init

In [6]:
# #TODO make saves a hash of 
# agent_name, config.tag, agent.task.name
# and timestamp

In [7]:
cuda = torch.cuda.is_available()
env_name = 'sonic256'
z_dim = 512  # latent dimensions
channels = 3*4

# RNN
action_dim = 10
image_size = 128

verbose = True  # Set this true to render (and make it go slower)

NAME = 'RNN_v3b_128im_512z_1512_v6i_VAE5_all'
ppo_save_file = './outputs/{NAME}/PPO_512z_all_f.pkl'.format(NAME=NAME)

if not os.path.isdir('./outputs/{NAME}'.format(NAME=NAME)):
    os.makedirs('./outputs/{NAME}'.format(NAME=NAME))

# Log to file and stream
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(NAME)

log_dir = log_dir='./outputs/{NAME}'.format(NAME=NAME)
print(log_dir)

deep_rl_logger = get_logger(
    NAME,
    file_name='deep_rl_ppo.log',
    level=logging.INFO,
    log_dir='./outputs/{NAME}'.format(NAME=NAME), )

./outputs/RNN_v3b_128im_512z_1512_v6i_VAE5_all


# World model

In [8]:
# Load VAE
# TODO swap z and k dim, since it's inconsistent with other models
vae = VAE5(image_size=image_size, z_dim=128, conv_dim=64, code_dim=8, k_dim=z_dim, channels=channels)
    
# Load MDRNN
action_dim, hidden_size, n_mixture, temp = action_dim, z_dim*2, 5, 0.0

mdnrnn = MDNRNN2(z_dim, action_dim, hidden_size, n_mixture, temp)
    
finv = InverseModel(z_dim, action_dim, hidden_size=z_dim*2)
    
world_model = WorldModel(vae, mdnrnn, finv, logger=deep_rl_logger, lambda_vae_kld=1 / 4., C=0, lambda_finv=20, lambda_vae=1 / 20.)
world_model = world_model.train()
if cuda:
    world_model = world_model.cuda()

In [9]:
import torch.optim.lr_scheduler
torch.cuda.empty_cache()
optimizer = optim.Adam(world_model.parameters(), lr=1e-4)

world_model.optimizer = optimizer

# Train

In [12]:
z_state_dim=world_model.mdnrnn.z_dim + world_model.mdnrnn.hidden_size  + world_model.mdnrnn.action_dim


def task_fn(log_dir):
    return SonicWorldModelDeepRL(
        env_fn=lambda: RandomGameReset(make_env(
            'sonic', max_episode_steps=1000, to_gray=False, image_size=image_size)),
        log_dir=log_dir,
        verbose=verbose
    )

config = Config()

verbose = False  # Set this true to render (and make it go slower)
config.num_workers = 1 if verbose else 8
config.task_fn = lambda: ParallelizedTask(
    task_fn, config.num_workers, single_process=config.num_workers == 1)
config.optimizer_fn = lambda params: torch.optim.Adam(params, 3e-4, eps=1e-5)
config.network_fn = lambda state_dim, action_dim: CategoricalWorldActorCriticNet(
    state_dim, action_dim, FCBody(z_state_dim, hidden_units=(64, 64), gate=F.relu), gpu=0 if cuda else -1, world_model_fn=lambda: world_model,
    render=(config.num_workers==1 and verbose),
    z_shape=(32, 16)
)
config.discount = 0.99
config.logger = deep_rl_logger
config.use_gae = True
config.gae_tau = 0.95
config.entropy_weight = 0.001
config.gradient_clip = 0.4
config.rollout_length = 64//config.num_workers
config.optimization_epochs = 10
config.num_mini_batches = 8
config.ppo_ratio_clip = 0.2
config.iteration_log_interval = 10
agent = PPOAgent(config)

print('rollout of ', config.rollout_length*config.num_workers)
print('mini batch', (config.rollout_length*config.num_workers)//config.num_mini_batches)


if os.path.isfile(ppo_save_file):
    print('loading ppo_save_file', ppo_save_file, 'modified', time.ctime(os.path.getmtime(ppo_save_file)))
    agent.load(ppo_save_file)
else:
    print("couldn't find save")


game: SonicAndKnuckles3-Genesis state: CarnivalNightZone.Act1
game: SonicAndKnuckles3-Genesis state: SandopolisZone.Act1
game: SonicAndKnuckles3-Genesis state: LavaReefZone.Act2
game: SonicAndKnuckles3-Genesis state: MushroomHillZone.Act2
game: SonicTheHedgehog2-Genesis state: WingFortressZone
game: SonicAndKnuckles3-Genesis state: LaunchBaseZone.Act1
game: SonicAndKnuckles3-Genesis state: HiddenPalaceZone
game: SonicTheHedgehog2-Genesis state: ChemicalPlantZone.Act1
rollout of  64
mini batch 8
loading ppo_save_file ./outputs/RNN_v3b_128im_512z_1512_v6i_VAE5_all/PPO_512z_all_f.pkl modified Mon Jun  4 13:38:00 2018


RuntimeError: Error(s) in loading state_dict for CategoricalWorldActorCriticNet:
	While copying the parameter named "network.phi_body.layers.0.weight", whose dimensions in the model are torch.Size([64, 1546]) and whose dimensions in the checkpoint are torch.Size([64, 1536]).

Process ProcessWrapper-11:
Process ProcessWrapper-14:
Process ProcessWrapper-13:
Process ProcessWrapper-10:
Process ProcessWrapper-16:
Process ProcessWrapper-9:
Process ProcessWrapper-15:
Traceback (most recent call last):
Traceback (most recent call last):
Process ProcessWrapper-12:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/wassname/.pyenv/versions/3.5.3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/wassname/.pyenv/versions/3.5.3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/wassname/.pyenv/versions/3.5.3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/media/oldhome/wassname/Documents/projects/retro_sonic_comp/DeepRL/deep_rl/component/task.py", line 177, in run
    

In [ ]:
try:
    run_iterations(agent, log_dir=log_dir)
except:
    if config.num_workers == 1:
        agent.task.tasks[0].env.close()
    else:
        [t.close() for t in agent.task.tasks]
    print("saving", ppo_save_file)
    agent.save(ppo_save_file)

    # Backup since it sometimes get's corrupted
    ts = datetime.datetime.utcnow().strftime('%Y%m%d_%H-%M-%S')
    print("saving backup",
          ppo_save_file.replace('.pkl', '-%s.pkl' % ts),)
    agent.save(ppo_save_file.replace('.pkl', '-%s.pkl' % ts))
    raise

2018-06-04 13:38:44,793 - RNN_v3b_128im_512z_1512_v6i_VAE5_all - INFO: loss_rnn=18.1569, loss_inv= 14.6788=20.0000 * 0.7339, loss_vae=1402.5749=0.0500 * (28039.4980 + 0.2500 * 47.9971)
2018-06-04 13:38:44,795 - RNN_v3b_128im_512z_1512_v6i_VAE5_all - INFO: total steps 64, min/mean/max reward 0.0000/0.0000/0.0000 of 8
2018-06-04 13:38:44,796 - RNN_v3b_128im_512z_1512_v6i_VAE5_all - INFO: running min/mean/max reward 0.0000/0.0000/0.0000 of 8 0.3846 s/rollout
2018-06-04 13:39:15,216 - RNN_v3b_128im_512z_1512_v6i_VAE5_all - INFO: loss_rnn=17.9926, loss_inv= 14.6899=20.0000 * 0.7345, loss_vae=891.3454=0.0500 * (17786.1480 + 0.2500 * 163.0382)
2018-06-04 13:39:15,217 - RNN_v3b_128im_512z_1512_v6i_VAE5_all - INFO: total steps 704, min/mean/max reward 0.0000/0.0000/0.0000 of 8
2018-06-04 13:39:15,218 - RNN_v3b_128im_512z_1512_v6i_VAE5_all - INFO: running min/mean/max reward 0.0000/0.0000/0.0000 of 88 0.3807 s/rollout
2018-06-04 13:39:44,896 - RNN_v3b_128im_512z_1512_v6i_VAE5_all - INFO: loss_rn

saving ./outputs/RNN_v3b_128im_512z_1512_v6i_VAE5_all/PPO_512z_all_f.pkl



To monitor with tensorboard
```sh
cd ~/Documents/projects/retro_sonic_comp/world-models-pytorch/log 
tensorboard  --logdir .
#then open http://localhost:6006/#scalars
```


In [ ]:
agent.save(ppo_save_file)
ppo_save_file

# summarize

In [ ]:
from IPython.display import display

with torch.no_grad():
    img = np.random.randn(image_size, image_size, 3)
    action = np.array(np.random.randint(0,action_dim))[np.newaxis]
    action = Variable(torch.from_numpy(action)).float().cuda()[np.newaxis]
    gpu_img = Variable(torch.from_numpy(img[np.newaxis].transpose(0, 3, 1, 2))).float().cuda()
    if cuda:
        gpu_img = gpu_img.cuda()
    with TorchSummarizeDf(vae) as tdf:
        x, mu_vae, logvar_vae = vae.forward(gpu_img)
        z = vae.sample(mu_vae, logvar_vae)
        df_vae = tdf.make_df()

    display(df_vae[df_vae.level<2])
    
    with TorchSummarizeDf(mdnrnn) as tdf: 
        pi, mu, sigma, hidden_state = mdnrnn.forward(z.unsqueeze(1).repeat((1,2,1)))
        z_next = mdnrnn.sample(pi, mu, sigma)
        df_mdnrnn = tdf.make_df()
    
    display(df_mdnrnn)
    

    with TorchSummarizeDf(finv) as tdf:
        finv(z.repeat((1,2,1)), z_next)   
        df_finv = tdf.make_df()
    display(df_finv)

    with TorchSummarizeDf(world_model) as tdf:
        world_model(gpu_img, action)
        df_world_model = tdf.make_df()
    display(df_world_model[df_world_model.level<2])
    
    del img, action, gpu_img, x, mu, z, z_next, mu_vae, pi, sigma, logvar_vae